# Am I the Asshole EDA
This file will be dedicated to explore how to craft responses to posts in the subreddit r/AmITheAsshole. 

In [49]:
import os 
import sys
import importlib
from pathlib import Path
from dotenv import load_dotenv
from google.cloud import bigquery

In [50]:
sys.path.append("..")

# load enviornment variables for praw to work later
load_dotenv(dotenv_path=Path("../app_env_vars.env"))

True

In [19]:
client = bigquery.Client()

## Create Dataset
Using Google Bigquery, create datasets for the data that is about to be loaded. The dataset will be called **AITA_dataset**

In [20]:
PROJ_NAME = "bonion"
DATASET_NAME = "AITA_dataset"

#### Get all dataset ID's
To prevent overwriting pre-existing datasets, get all dataset id's to check before any creation step.

In [34]:
# Get all datasets 
datasets = list(client.list_datasets())  # Make an API request.
dataset_ids = [dataset.dataset_id for dataset in datasets]
project = client.project

if datasets:
    print("Datasets in project {}:".format(project))
    for dataset in datasets:
        print("\t{}".format(dataset.dataset_id))
else:
    print("{} project does not contain any datasets.".format(project))

Datasets in project bonion:
	AITA_dataset


In [25]:
# create dataset
if DATASET_NAME not in dataset_ids:
    dataset = bigquery.Dataset("{}.{}".format(PROJ_NAME, DATASET_NAME))
    dataset.location = "US"

    # send dataset to API for completion
    initial_post_dataset = client.create_dataset(dataset, timeout=30)  # Make an API request.
    print("Created dataset {}.{}".format(client.project, dataset.dataset_id))

## Post Table
The initial post for each of the comments will need to be stored to reference its content which dictated the reply.

In [39]:
# table id definitions
post_table_id = "{}.{}.post_table".format(PROJ_NAME, DATASET_NAME)
post_reply_table_id = "{}.{}.post_reply_table".format(PROJ_NAME, DATASET_NAME)
post_reply_top_children_table_id = "{}.{}.post_reply_top_children".format(PROJ_NAME, DATASET_NAME)

#### Get all Table ID's
To prevent overwriting pre-existing tables, get all table id's to check before any creation step.

In [37]:
tables = list(client.list_tables(DATASET_NAME))  # Make an API request.
table_ids = [table.table_id for table in tables]

print("Tables contained in '{}':".format(DATASET_NAME))
for table in tables:
    print("{}.{}.{}".format(table.project, table.dataset_id, table.table_id))

Tables contained in 'AITA_dataset':
bonion.AITA_dataset.init_post_table


In [36]:
if post_table_id not in tables: 
    schema = [
        bigquery.SchemaField("reddit_post_id", "STRING", mode="REQUIRED"),
        bigquery.SchemaField("post_self_text", "STRING", mode="REQUIRED"),
        bigquery.SchemaField("upvotes", "INTEGER", mode="REQUIRED"),
        bigquery.SchemaField("num_responses", "INTEGER", mode="REQUIRED")
    ]

    post_table = bigquery.Table(post_table_id, schema=schema)
    post_table.description = """
        A table which holds popular posts from the subreddit r/AITA
    """

    table = client.create_table(post_table)
    print(
        "Created table {}.{}.{}".format(table.project, table.dataset_id, table.table_id)
    )

Created table bonion.AITA_dataset.init_post_table


## Post Reply Table
This table holds all of the responses for each of the posts in the post table.

In [ ]:
if post_reply_table_id not in tables: 
    schema = [
        bigquery.SchemaField("reddit_post_id", "STRING", mode="REQUIRED"),
        bigquery.SchemaField("upvotes", "INTEGER", mode="REQUIRED"),
        bigquery.SchemaField("num_responses", "INTEGER", mode="REQUIRED")
    ]

    post_reply_table = bigquery.Table(post_reply_table_id, schema=schema)
    post_reply_table.description = """
        A table which holds the most popular replys to saved posts from the subreddit r/AITA
    """

    table = client.create_table(post_reply_table)
    print(
        "Created table {}.{}.{}".format(table.project, table.dataset_id, table.table_id)
    )

## Post Reply Top Children
This table will hold replys to the initial post reply. This will be used for also dictating the performance of a reply. Because there can be infinite replies to any given post or reply, each post reply id will be limited to some number of replies. These replies will likley be based off of the same metrics as the original post for "quality". 

In [ ]:
if post_reply_top_children_table_id not in tables: 
    schema = [
        bigquery.SchemaField("reddit_post_id", "STRING", mode="REQUIRED"),
        bigquery.SchemaField("upvotes", "INTEGER", mode="REQUIRED"),
        bigquery.SchemaField("num_responses", "INTEGER", mode="REQUIRED")
    ]

    post_reply_top_children_table = bigquery.Table(post_reply_top_children_table_id, schema=schema)
    post_reply_top_children_table.description = """
        This table will hold replys to the initial post reply. 
        This will be used for also dictating the performance of a reply. 
        Because there can be infinite replies to any given post or reply, 
        each post reply id will be limited to some number of replies. 
        These replies will likley be based off of the same metrics as 
        the original post for "quality". 
    """

    table = client.create_table(post_reply_top_children_table)
    print(
        "Created table {}.{}.{}".format(table.project, table.dataset_id, table.table_id)
    )

## Collect Data
Using my PrawInstance Object, I will be collecting the top posts from the subreddit. I will be storing this data in google bigquery.

In [46]:
import praw_instance
importlib.reload(praw_instance)

<module 'praw_instance' from '/home/cstainsby/class/dataProj/bonion/src/notebooks/../praw_instance.py'>

In [51]:
praw_inst = praw_instance.PrawInstance()